In [1]:
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm
import shutil

# Ruta de la carpeta de muestras
ruta_muestras = r'D:\Python\TESIS\MUESTRAS_CENTRADAS_V2\MUESTRAS_C12_450MHZ'

# Ruta de la carpeta de salida
ruta_salida = r'D:\Python\TESIS\CLUSTERS_C12_CALC_K'

# Crear carpeta de salida si no existe
if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

# Cargar y preprocesar las imágenes (sin redimensionar)
imagenes = []
nombres_imagenes = []

# Obtener la lista de archivos en la carpeta de muestras
archivos = os.listdir(ruta_muestras)

# Leer y procesar cada imagen
for archivo in tqdm(archivos, desc='Cargando imágenes'):
    ruta_imagen = os.path.join(ruta_muestras, archivo)
    try:
        imagen = imread(ruta_imagen, as_gray=True)  # Leer en escala de grises
        imagen = resize(imagen, (170, 170))  # Asegurar que todas las imágenes tengan el tamaño 170x170
        imagen_vector = imagen.flatten()  # Convertir la imagen en un vector 1D
        imagenes.append(imagen_vector)
        nombres_imagenes.append(archivo)
    except Exception as e:
        print(f'Error al procesar {archivo}: {e}')

X = np.array(imagenes)
print(f'Total de imágenes procesadas: {X.shape[0]}')

# Aplicar PCA para reducir la dimensionalidad a, por ejemplo, 50 componentes principales
pca = PCA(n_components=75)
X_pca = pca.fit_transform(X)

print(f"Dimensiones reducidas: {X_pca.shape}")

# Definir K-Means con K=9
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X_pca)

# Obtener etiquetas de los clusters
labels = kmeans.labels_

# Crear subcarpetas para cada cluster dentro de la carpeta de salida
for cluster in range(4):
    cluster_path = os.path.join(ruta_salida, f'cluster_{cluster}')
    if not os.path.exists(cluster_path):
        os.makedirs(cluster_path)

# Mover cada imagen a la carpeta de su cluster correspondiente
for i, archivo in enumerate(nombres_imagenes):
    cluster_label = labels[i]
    cluster_path = os.path.join(ruta_salida, f'cluster_{cluster_label}')
    ruta_origen = os.path.join(ruta_muestras, archivo)
    ruta_destino = os.path.join(cluster_path, archivo)
    shutil.copy(ruta_origen, ruta_destino)

print("Clusterización completada y resultados guardados en las carpetas correspondientes.")


Cargando imágenes: 100%|████████████████████████████████████████████████████████████| 827/827 [00:03<00:00, 221.78it/s]


Total de imágenes procesadas: 827
Dimensiones reducidas: (827, 75)


C:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Clusterización completada y resultados guardados en las carpetas correspondientes.
